In [9]:
import pandas as pd
import matplotlib.pyplot as plt

data_dir = '../data' 
articles_df = pd.read_csv(f"{data_dir}/rating.csv") 

website_article_counts = articles_df['source_name'].value_counts()

top_ten_websites = website_article_counts.head(10)
bottom_ten_websites = website_article_counts.tail(10)

print("Top ten websites with the largest counts of news articles: ")
print(top_ten_websites)
print("\nBottom ten websites with the smallest counts of news articles: ")
print(bottom_ten_websites)


Top ten websites with the largest counts of news articles: 
source_name
ETF Daily News        16746
The Times of India     7504
GlobeNewswire          5423
Globalsecurity.org     3119
Forbes                 2784
BBC News               2113
ABC News               2058
Business Insider       2034
The Punch              1800
Al Jazeera English     1664
Name: count, dtype: int64

Bottom ten websites with the smallest counts of news articles: 
source_name
CNA                            674
Time                           600
Android Central                522
Gizmodo.com                    388
ReadWrite                      324
Euronews                       286
Wired                          270
CNN                            267
The Verge                      214
AllAfrica - Top Africa News     20
Name: count, dtype: int64


In [11]:
import pandas as pd

# Load the traffic data
data_dir = '../data'  # Adjust path as per your directory structure
traffic_df = pd.read_csv(f"{data_dir}/traffic.csv")

# Calculate traffic counts for each website
website_traffic_counts = traffic_df.groupby('Domain')['GlobalRank'].min()

# Get the top and bottom ten websites
top_ten_websites_traffic = website_traffic_counts.nlargest(10)
bottom_ten_websites_traffic = website_traffic_counts.nsmallest(10)

# Print top ten websites with highest traffic
print("Top Ten Websites with the Highest Traffic:")
print(top_ten_websites_traffic)

# Print bottom ten websites with lowest traffic
print("\nBottom Ten Websites with the Lowest Traffic:")
print(bottom_ten_websites_traffic)


Top Ten Websites with the Highest Traffic:
Domain
toyotamusicfactory.com    1000000
soderhomes.com             999999
pinkwater.com              999998
mt-lock.com                999997
kireie.com                 999996
keith-baker.com            999995
irishcycle.com             999994
hmag.com                   999993
exploring-africa.com       999992
eiretrip.com               999991
Name: GlobalRank, dtype: int64

Bottom Ten Websites with the Lowest Traffic:
Domain
google.com               1
facebook.com             2
youtube.com              3
twitter.com              4
instagram.com            5
linkedin.com             6
apple.com                7
microsoft.com            8
googletagmanager.com     9
wikipedia.org           10
Name: GlobalRank, dtype: int64


In [12]:
import pandas as pd

# Load the domain location data
data_dir = '../data'  # Adjust path as per your directory structure
domains_location_df = pd.read_csv(f"{data_dir}/domains_location.csv")

# Calculate the count of domains for each country
country_domain_counts = domains_location_df['Country'].value_counts()

# Get the top and bottom ten countries
top_ten_countries = country_domain_counts.head(10)
bottom_ten_countries = country_domain_counts.tail(10)

# Print top ten countries with the highest number of news media organizations
print("Top Ten Countries with the Highest Number of News Media Organizations:")
print(top_ten_countries)

# Print bottom ten countries with the lowest number of news media organizations
print("\nBottom Ten Countries with the Lowest Number of News Media Organizations:")
print(bottom_ten_countries)


Top Ten Countries with the Highest Number of News Media Organizations:
Country
United States     14111
United Kingdom     1950
Italy              1810
France             1041
Russia             1024
Canada              887
Germany             884
China               780
Turkey              725
India               686
Name: count, dtype: int64

Bottom Ten Countries with the Lowest Number of News Media Organizations:
Country
Greenland         1
Guernsey          1
Isle of Man       1
Cook Islands      1
Guinea-Bissau     1
Micronesia        1
Aruba             1
American Samoa    1
Guadeloupe        1
Saint Helena      1
Name: count, dtype: int64


In [15]:
import pandas as pd
import re

# Load the data containing articles
data_dir = '../data'  # Adjust path as per your directory structure
articles_df = pd.read_csv(f"{data_dir}/rating.csv")  # Assuming the dataset is in CSV format

# Extract mentions of countries from the article content
countries = ["Africa", "US", "China", "EU", "Russia", "Ukraine", "Middle East"]  # List of countries to search for

# Initialize counts for each country
country_counts = {country: 0 for country in countries}

# Iterate over each article and count mentions of countries
for content in articles_df['content']:
    for country in countries:
        # Count occurrences of the country in the content
        country_counts[country] += len(re.findall(r'\b{}\b'.format(country), str(content), re.IGNORECASE))

# Convert counts to DataFrame for easier manipulation
country_counts_df = pd.DataFrame.from_dict(country_counts, orient='index', columns=['Count'])

# Get the top and bottom ten countries with many articles written about them
top_ten_countries_articles = country_counts_df.nlargest(10, 'Count')
bottom_ten_countries_articles = country_counts_df.nsmallest(10, 'Count')

# Print top ten countries with many articles written about them
print("Top Ten Countries with Many Articles Written About Them:")
print(top_ten_countries_articles)

# Print bottom ten countries with few articles written about them
print("\nBottom Ten Countries with Few Articles Written About Them:")
print(bottom_ten_countries_articles)


Top Ten Countries with Many Articles Written About Them:
             Count
US            2321
China         1062
Ukraine       1023
Russia         733
Africa         488
EU             283
Middle East    186

Bottom Ten Countries with Few Articles Written About Them:
             Count
Middle East    186
EU             283
Africa         488
Russia         733
Ukraine       1023
China         1062
US            2321


In [18]:
import pandas as pd

# Load the data containing information about websites and reported countries
data_dir = '../data'  # Adjust path as per your directory structure
articles_df = pd.read_csv(f"{data_dir}/rating.csv")  # Assuming the dataset is in CSV format

# Define the African countries
african_countries = ['Algeria', 'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cabo Verde', 'Cameroon', 'Central African Republic', 'Chad', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', "Cote d'Ivoire", 'Djibouti', 'Egypt', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Morocco', 'Mozambique', 'Namibia', 'Niger', 'Nigeria', 'Rwanda', 'Sao Tome and Principe', 'Senegal', 'Seychelles', 'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Tanzania', 'Togo', 'Tunisia', 'Uganda', 'Zambia', 'Zimbabwe']

# Initialize counts for each African country
african_country_counts = {country: 0 for country in african_countries}

# Iterate over each article and count websites reporting news content for each African country
for index, row in articles_df.iterrows():
    for country in african_countries:
        # Check if the country is mentioned in the article content
        if country.lower() in row['content'].lower():
            african_country_counts[country] += 1

# Convert counts to DataFrame for easier manipulation
african_country_counts_df = pd.DataFrame.from_dict(african_country_counts, orient='index', columns=['Count'])

# Get the top and bottom ten African countries with websites reporting news content
top_ten_african_countries = african_country_counts_df.nlargest(10, 'Count')
bottom_ten_african_countries = african_country_counts_df.nsmallest(10, 'Count')

# Print top ten African countries with websites reporting news content
print("Top Ten African Countries with Websites Reporting News Content:")
print(top_ten_african_countries)

# Print bottom ten African countries with websites reporting news content
print("\nBottom Ten African Countries with Websites Reporting News Content:")
print(bottom_ten_african_countries)


Top Ten African Countries with Websites Reporting News Content:
              Count
Niger           805
Nigeria         674
South Africa    297
Egypt           167
Mali            148
Kenya           132
Sudan           113
Ethiopia         62
Morocco          55
Liberia          54

Bottom Ten African Countries with Websites Reporting News Content:
                       Count
Congo (Brazzaville)        0
Congo (Kinshasa)           0
Sao Tome and Principe      0
Burundi                    1
Cabo Verde                 1
Comoros                    1
Cote d'Ivoire              1
Eswatini                   1
Mauritania                 1
Djibouti                   2
